In [1]:
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from PIL import Image
from numpy import savez_compressed
from numpy import asarray
from os import listdir
from mtcnn.mtcnn import MTCNN
from io import BytesIO
from keras.models import load_model
from sklearn.preprocessing import Normalizer
import pickle

#Method to extract Face
def extract_image(image):
    print(image)
    img1 = Image.open(image)            #open the image
    img1 = img1.convert('RGB')          #convert the image to RGB format
    pixels = asarray(img1)              #convert the image to numpy array
    detector = MTCNN()                  #assign the MTCNN detector
    f = detector.detect_faces(pixels)
    #fetching the (x,y)co-ordinate and (width-->w, height-->h) of the image
    x1,y1,w,h = f[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2 = abs(x1+w)
    y2 = abs(y1+h)
    #locate the co-ordinates of face in the image
    store_face = pixels[y1:y2,x1:x2]
    plt.imshow(store_face)
    image1 = Image.fromarray(store_face,'RGB')    #convert the numpy array to object
    image1 = image1.resize((160,160))             #resize the image
    face_array = asarray(image1)                  #image to array
    return face_array

def extract_image_base64(encoded_string):
    file = "data:image/png;base64,{}".format(encoded_string)
    starter = file.find(',')
    image_data = file[starter+1:]
    image_data = bytes(image_data, encoding="ascii")
#     print(image)
    img1 = Image.open(BytesIO(base64.b64decode(image_data)))           #open the image
    img1 = img1.convert('RGB')          #convert the image to RGB format
    pixels = asarray(img1)              #convert the image to numpy array
    detector = MTCNN()                  #assign the MTCNN detector
    f = detector.detect_faces(pixels)
    #fetching the (x,y)co-ordinate and (width-->w, height-->h) of the image
    x1,y1,w,h = f[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2 = abs(x1+w)
    y2 = abs(y1+h)
    #locate the co-ordinates of face in the image
    store_face = pixels[y1:y2,x1:x2]
#     plt.imshow(store_face)
    image1 = Image.fromarray(store_face,'RGB')    #convert the numpy array to object
    image1 = image1.resize((160,160))             #resize the image
    face_array = asarray(image1)                  #image to array
    return face_array


# extracting embeddings
def extract_embeddings(model, face_pixels):
    face_pixels = face_pixels.astype('float32')
    mean = face_pixels.mean()
    std = face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    samples = np.expand_dims(face_pixels, axis=0)
    yhat = model.predict(samples)
    return yhat[0]

In [2]:
import os

In [3]:
len(os.listdir('data/cv/indian_example/Indian-celebrities/'))

101

### image to base64

In [4]:
# For both Python 2.7 and Python 3.x
import base64
with open("/Users/wegzheng/Python_Projects/97_learn/data/cv/WechatIMG2074.jpeg", "rb") as fh:
    encoded_string = base64.b64encode(fh.read()).decode('utf-8')

In [5]:
print(encoded_string)

...


In [6]:
with open("/Users/wegzheng/Python_Projects/97_learn/data/cv/Aamir_Khan_2.jpg", "rb") as fh:
    encoded_string = base64.b64encode(fh.read()).decode('utf-8')
print(encoded_string)

...


### base64 to image

In [7]:
file = "data:image/png;base64,{}".format(encoded_string)
starter = file.find(',')
image_data = file[starter+1:]
image_data = bytes(image_data, encoding="ascii")
im = Image.open(BytesIO(base64.b64decode(image_data))).convert('RGB')

In [8]:
image_data

b'...'

In [9]:
face_array = extract_image_base64(encoded_string)

In [10]:
testx = np.asarray(face_array)
testx = testx.reshape(-1, 160, 160, 3)
print("Input test data shape: ", testx.shape)

Input test data shape:  (1, 160, 160, 3)


In [11]:
# find embeddings
model = load_model('data/cv/indian_example/facenet_keras.h5')
new_testx = list()
for test_pixels in testx:
    embeddings = extract_embeddings(model, test_pixels)
    new_testx.append(embeddings)
new_testx = asarray(new_testx)
print("Input test embedding shape: ", new_testx.shape)

Input test embedding shape:  (1, 128)


In [12]:
# normalize the input data
in_encode = Normalizer(norm='l2')
new_testx = in_encode.transform(new_testx)

In [13]:
# creating and saving some model
with open('model/cv_clf.pickle', 'rb') as f:
    clf = pickle.load(f)
    print('Model loaded')

Model loaded


In [14]:
predict_test = clf.predict(new_testx)

In [15]:
predict_test

array([0])

In [16]:
with open('model/out_encoder.pickle', 'rb') as f:
    out_encode = pickle.load(f)
    print("Encoder loaded")

Encoder loaded


In [17]:
print(out_encode.inverse_transform(predict_test))

['Aamir_Khan']


In [18]:
import requests

In [19]:
url = "https://www.aceshowbiz.com/images/photo/aamir_khan.jpg"
response = requests.get(url)

In [21]:
b64response = base64.b64encode(response.content).decode('utf-8')

In [22]:
b64response

'...'

In [23]:
url2 = "http://ai.suiteng.gzmetro.com/DescribeFileListNoS3?path=%2Fdata%2Fti-platform-fs%2Fdatalabel%2F0913%2F13686042735134908417%2FDisha_Patani_14.jpg"
response = requests.get(url2)
b64response = base64.b64encode(response.content).decode('utf-8')

In [27]:
response.content

b'{"Response":{"Error":{"Message":"Missing Ti tokens in request","Code":"AuthFailure.TokenFailure"},"RequestId":"277921807a22a636857498f794b29a94"}}\n'

In [24]:
b64response

'eyJSZXNwb25zZSI6eyJFcnJvciI6eyJNZXNzYWdlIjoiTWlzc2luZyBUaSB0b2tlbnMgaW4gcmVxdWVzdCIsIkNvZGUiOiJBdXRoRmFpbHVyZS5Ub2tlbkZhaWx1cmUifSwiUmVxdWVzdElkIjoiMjc3OTIxODA3YTIyYTYzNjg1NzQ5OGY3OTRiMjlhOTQifX0K'

In [25]:
uri = ("data:" + 
       response.headers['Content-Type'] + ";" +
       "base64," + base64.b64encode(response.content))

TypeError: can only concatenate str (not "bytes") to str

In [10]:
encoded_str